# Transformers for Sentiment Analysis

In [ ]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [ ]:
len(tokenizer.vocab)

In [ ]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

In [ ]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

In [ ]:
print(vars(train_data.examples[6]))

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

In [ ]:

LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# Build the Model


In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]

        return output

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

## Train the Model

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link ="https://drive.google.com/open?id=1MF2EAQLRhFd36EZOohImLUbUWAZzAkDm"

In [ ]:
fluff, id = link.split('=')
print (id) 

In [ ]:
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Academia.csv')  
df = pd.read_csv('.csv')
len(df)

In [ ]:
df = df.drop_duplicates("Text")

In [ ]:
df.head(-1)

In [ ]:
df = df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

In [ ]:
df = df.drop(columns=["Timestamp"])

In [ ]:
df["Count"] = 1

In [ ]:
df = pd.concat([df,df1])

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop_duplicates("Text")

In [ ]:
import numpy as np
df_split = np.array_split(df, 3)

In [ ]:
df.head(10)

In [ ]:
df["Count"] = 1

In [ ]:
df0 = df_split[0]
df0 = df0.reset_index(drop=True)

In [ ]:
sentiment0 = []
for i in range(len(df)):
  try:
    score = predict_sentiment(model, tokenizer, str(df["Text"][i]))
    sentiment0.append(score)
  except:
    sentiment0.append(0.5)
  i += 1


In [ ]:
sentiment0[1:10]

In [ ]:
df["Sentiment"] = sentiment0

In [ ]:
df["Sentiment"] = sentiment0

In [ ]:
df.groupby(["Dominant_Topic","Year"]).mean()

In [ ]:
df1 = pd.DataFrame(df1)

In [ ]:
y = cs["Count"]

In [ ]:
df.to_csv('keyword.csv')
from google.colab import files
files.download('keyword.csv')

In [ ]:
y = pd.DataFrame(y)

In [ ]:
df1 = df_split[1]
df1 = df1.reset_index(drop=True)

In [ ]:
sentiment1 = []
for i in range(len(df0)):
  try:
    score = predict_sentiment(model, tokenizer, str(df1["Text"][i]))
    sentiment1.append(score)
  except:
    sentiment1.append(0.5)
  i += 1


In [ ]:
sentiment2 = []
for i in range(len(df2)):
  try:
    score = predict_sentiment(model, tokenizer, str(df2["Text"][i]))
    sentiment2.append(score)
  except:
    sentiment2.append(0.5)
  i += 1


In [ ]:
df2["Sentiment"] = sentiment2

In [ ]:
len(sentiment)

In [ ]:
df.head()

In [ ]:
df.to_csv('Academia.csv')
from google.colab import files
files.download('Academia.csv')

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df = pd.concat([df0,df1,df2])

In [ ]:
len(df)

In [ ]:
df.head(-10)

In [ ]:
df["Sentiment"] = sentiment

In [ ]:
df.head(-1000)

In [ ]:
x = df.groupby(["Dominant_Topic"]).sum()

In [ ]:
x

In [ ]:
df.head()

In [ ]:
df.to_csv('Bert_Google.csv')
from google.colab import files
files.download('Bert_Google.csv')

In [ ]:
df7.head()

In [ ]:
df7 = df_split[7]
df7 = df7.reset_index(drop=True)
df7.head()

In [ ]:
sentiment = []
for i in range(len(df7)):
  try:
    score = predict_sentiment(model, tokenizer, df7["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1


In [ ]:
df7["Sentiment"] = sentiment

In [ ]:
df8 = df_split[8]
df8 = df8.reset_index(drop=True)
df8.head()

In [ ]:
sentiment = []
for i in range(len(df8)):
  try:
    score = predict_sentiment(model, tokenizer, df8["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1


In [ ]:
df8["Sentiment"] = sentiment

In [ ]:
df9 = df_split[9]
df9 = df9.reset_index(drop=True)
df9.head()

In [ ]:
sentiment = []
for i in range(len(df9)):
  try:
    score = predict_sentiment(model, tokenizer, df9["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1


In [ ]:
df9["Sentiment"] = sentiment

In [ ]:
df10 = df_split[10]
df10 = df10.reset_index(drop=True)
df10.head()

In [ ]:
sentiment = []
for i in range(len(df10)):
  try:
    score = predict_sentiment(model, tokenizer, df10["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1


In [ ]:
df10["Sentiment"] = sentiment

In [ ]:
df11 = df_split[11]
df11 = df11.reset_index(drop=True)
df11.head()

In [ ]:
sentiment = []
for i in range(len(df11)):
  try:
    score = predict_sentiment(model, tokenizer, df11["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df11["Sentiment"] = sentiment

In [ ]:
df0 = df_split[0]
df0 = df11.reset_index(drop=True)
df0.head()

In [ ]:
sentiment = []
for i in range(len(df0)):
  try:
    score = predict_sentiment(model, tokenizer, df0["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df0["Sentiment"] = sentiment

In [ ]:
df1 = df_split[1]
df1 = df1.reset_index(drop=True)
df1.head()

In [ ]:
sentiment = []
for i in range(len(df1)):
  try:
    score = predict_sentiment(model, tokenizer, df1["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df1["Sentiment"] = sentiment

In [ ]:
df2 = df_split[2]
df2 = df2.reset_index(drop=True)
df2.head()

In [ ]:
sentiment = []
for i in range(len(df2)):
  try:
    score = predict_sentiment(model, tokenizer, df2["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df2["Sentiment"] = sentiment

In [ ]:
df3 = df_split[3]
df3 = df3.reset_index(drop=True)
df3.head()

In [ ]:
sentiment = []
for i in range(len(df3)):
  try:
    score = predict_sentiment(model, tokenizer, df3["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df3["Sentiment"] = sentiment

In [ ]:
df4 = df_split[4]
df4 = df4.reset_index(drop=True)
df4.head()

In [ ]:
sentiment = []
for i in range(len(df4)):
  try:
    score = predict_sentiment(model, tokenizer, df4["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df4["Sentiment"] = sentiment

In [ ]:
df5 = df_split[5]
df5 = df5.reset_index(drop=True)
df5.head()

In [ ]:
sentiment = []
for i in range(len(df5)):
  try:
    score = predict_sentiment(model, tokenizer, df5["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
df5["Sentiment"] = sentiment

In [ ]:
df11 = df_split[11]
df11 = df11.reset_index(drop=True)
df11.head()

In [ ]:
sentiment = []
for i in range(len(df0)):
  try:
    score = predict_sentiment(model, tokenizer, df0["Text"][i])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

In [ ]:
import pandas as pd


In [ ]:
df = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11],ignore_index = True)

In [ ]:
len(df)

In [ ]:
df.to_csv('df.csv')
from google.colab import files
files.download('df.csv')

In [ ]:
df1.to_csv('df1.csv')
from google.colab import files
files.download('df1.csv')

In [ ]:
df2.to_csv('df2.csv')
from google.colab import files
files.download('df2.csv')

In [ ]:
df3.to_csv('df3.csv')
from google.colab import files
files.download('df3.csv')

In [ ]:
df4.to_csv('df4.csv')
from google.colab import files
files.download('df4.csv')

In [ ]:
df5.to_csv('df5.csv')
from google.colab import files
files.download('df5.csv')

In [ ]:
df6.to_csv('df6.csv')
from google.colab import files
files.download('df6.csv')

In [ ]:
df7.to_csv('df7.csv')
from google.colab import files
files.download('df7.csv')

In [ ]:
df08.to_csv('df8.csv')
from google.colab import files
files.download('df8.csv')

In [ ]:
df9.to_csv('df9.csv')
from google.colab import files
files.download('df9.csv')

In [ ]:
df10.to_csv('df10.csv')
from google.colab import files
files.download('df10.csv')

In [ ]:
df11.to_csv('df11.csv')
from google.colab import files
files.download('df11.csv')

In [ ]:
df.to_csv('df.csv')
from google.colab import files
files.download('df.csv')

In [ ]:
csgo = pd.read_csv("df.csv")

In [ ]:
df["Text"][0]

In [ ]:
df1["Sentiment"] = sentiment

In [ ]:
len(df1)

In [ ]:
df1.to_csv('LDA_Google.csv')
from google.colab import files
files.download('LDA_Google.csv')

In [ ]:
df1 = df_split[1]
df1.head()

In [ ]:

sentiment = []
for i in range(len(df1)):
  try:
    score = predict_sentiment(model, tokenizer, df1["Text"][i+48896])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df1["Sentiment"] = sentiment
df1.head()

In [ ]:
df2 = df_split[2]
df2.head()

In [ ]:

sentiment = []
for i in range(len(df2)):
  try:
    score = predict_sentiment(model, tokenizer, df2["Text"][i+97792])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df2["Sentiment"] = sentiment
df2.head()

In [ ]:
df3 = df_split[3]
df3.head()

In [ ]:

sentiment = []
for i in range(len(df3)):
  try:
    score = predict_sentiment(model, tokenizer, df3["Text"][i+146688])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df3["Sentiment"] = sentiment
df3.head()

In [ ]:
df4 = df_split[4]
df4.head()

In [ ]:

sentiment = []
for i in range(len(df4)):
  try:
    score = predict_sentiment(model, tokenizer, df4["Text"][i+195584])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df4["Sentiment"] = sentiment
df4.head()

In [ ]:
df5 = df_split[5]
df5.head()

In [ ]:

sentiment = []
for i in range(len(df5)):
  try:
    score = predict_sentiment(model, tokenizer, df5["Text"][i+244480])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df5["Sentiment"] = sentiment
df5.head()

In [ ]:
df6 = df_split[6]
df6.head()

In [ ]:

sentiment = []
for i in range(len(df6)):
  try:
    score = predict_sentiment(model, tokenizer, df6["Text"][i+293375])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df6["Sentiment"] = sentiment
df6.head()

In [ ]:
df7 = df_split[7]
df7.head()

In [ ]:

sentiment = []
for i in range(len(df7)):
  try:
    score = predict_sentiment(model, tokenizer, df7["Text"][i+342270])
    sentiment.append(score)
  except:
    sentiment.append(0.5)
  i += 1

df7["Sentiment"] = sentiment
df7.head()

In [ ]:
len(df1)

In [ ]:
len(df2)

In [ ]:
df1.to_csv('LDA_Bing.csv')

In [ ]:
df.to_csv('Twitter Sentiment.csv')
from google.colab import files
files.download('Twitter Sentiment.csv')

In [ ]:

dfc = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7])

In [ ]:
#twitter.head()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("LDA_Twitter.csv")

In [ ]:
df.groupby("Dominant_Topic").mean()